In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import plotly.express as px

# パスの設定
PATH_HEAD = r'C:\Users\猛\Desktop\yoshiki_study\cross_section_ensemble'
TARGET_FILE = r'C:\Users\猛\Desktop\yoshiki_study\cross_section_ensemble\Result\simulation\210711_205153\all_result.pickle'
os.chdir(PATH_HEAD)

# 自作モジュール
from _simulation.util_simulation import convert_df
from _simulation.simulator import Simulator
from utils.interpreter import Interpreter

## 各トレーダーの投資戦略の可視化

In [ ]:
from _simulation.experimenter import Experimenter

ex = Experimenter(hyperparms_company_common)
X_train, X_test, y_train, y_test = ex.set_simulation(dict_output, train_num=simulation_setting['train_num'])
weight_test, df_params, df_info, traders_performance_log = ex.learn(X_train, y_train, X_test, y_test)       
traders_weight_test = ex.tc.predict_traders(X_test, ticker_info={})

traders_return = ex.tc.calc_return_traders(traders_weight_test, y_test)
traders_return = pd.DataFrame(traders_return)
traders_return.columns = ['trader'+str(i) for i in range(hyperparms_company_common['N_trader'])]

In [ ]:
from utils.cluster import cluster_KMeans_base

df_params_extracted = df_params[df_params['ensemble_weight']>0]
df_params['adopted'] = df_params['ensemble_weight']>0
traders_ID_extracted = df_params_extracted.loc[:, 'trader_ID'].unique()
traders_extracted = traders_return.iloc[:,list(traders_ID_extracted)]
corr = np.corrcoef(traders_extracted.T)
corr1, clusters, silh = cluster_KMeans_base(corr, 
                                            hyperparms_company_common['max_num_clusters'],
                                            hyperparms_company_common['n_init_clusters'], 
                                            hyperparms_company_common['random_seed'])

sns.heatmap(corr1, vmax=1, vmin=-1, center=0)